# Data Collection, Cleansing and Storage

1) Preliminary Setup
2) Download Data from FRED API
3) Cleanse Data by web scraping for automatical update
4) Save Data into Google BigQuery

# 1. Preliminary Setup

In [1]:
# pyfredapi need API Key which can be applied from below website
# https://fred.stlouisfed.org/docs/api/api_key.html

# install api for access financial data from FRED website https://fred.stlouisfed.org/
!pip install pyfredapi --upgrade --quiet

# install api for access google bigquery
!pip install google-cloud-bigquery --quiet

In [2]:
# import api for access financial data 
import pyfredapi as pf
# import pandas for data manipulation
import pandas as pd
# import numpy for data manipulation
import numpy as np
# import data list from another notebook
import Data_List

# import requests for web scraping
import requests
# import BeautifulSoup for parsing html
from bs4 import BeautifulSoup

# import api for access google bigquery
from google.cloud import bigquery as bq
# import for getting details for authentication and project id
import google.auth

# 2. Download Data from FRED API

In [3]:
# Define lambda function for simple data cleanse
get = lambda series_id: pf.get_series(series_id)[["date","value"]].set_index("date").rename({"value":series_id},axis=1).dropna()

In [4]:
# call the data list from another .py
dict_data = Data_List.dict_data

# define a dictionary for store the dataframe of various financial data
dict_df = {}
for data in dict_data:
    dict_df[data] = get(data)

# 3. Cleanse Data by web scraping for automatical update

In [5]:
# create a dataframe to join all economic data to fedfunds to find any na of economic data
combine = pd.DataFrame()

for df in dict_df:
    if combine.empty:
        combine = dict_df[df].copy()
    else:
        combine = combine.join(dict_df[df])

In [6]:
# FEDFUNDS is available on 1st of next month

# TB3MS is available on 1st of next month

# CPIAUCSL is available near 12th of next month
# CPIAUCSL forecast is available after last data is released

# UNRATE is available near 8th of next month
# UNRATE forecast is available after last data is released

# GDP (advance estimate) is available near 28th of next month of quarter
# GDP (second estimate) is available near 28th of next second month of quarter
# GDP (third estimate) is available near 28th of next third month of quarter
# GDP forecast is available near 28th of first month of current quarter

In [7]:
# get forecast data for unrate
# web scraping from https://tradingeconomics.com/united-states/unemployment-rate
def unrate_forecast():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    result = requests.get('https://tradingeconomics.com/united-states/unemployment-rate', headers=headers)
    soup = BeautifulSoup(result.text, 'html.parser')

    css_unrate_forecast = 'td.datatable-item.datatable-item-positive'
    css_unrate_actual = css_unrate_forecast + '#actual'
    unrate_forcast = soup.select(css_unrate_forecast)[-1].text.split()
    unrate_actual = soup.select(css_unrate_actual)[-1].text.split()
    unrate_data = unrate_forcast if not unrate_actual else unrate_actual

    css_unrate_forecast_date = 'tr.an-estimate-row > td[style="text-align: left; padding-left: 10px; white-space: nowrap;"]'
    unrate_forecast_date = pd.to_datetime(soup.select(css_unrate_forecast_date)[1].text[:7] + '-01')

    css_unrate_publish_date = css_unrate_forecast_date
    unrate_publish_date = soup.select(css_unrate_forecast_date)[-1].text

    return unrate_forecast_date, float(unrate_data[0][:-1]), unrate_publish_date

In [8]:
# get forecast data of cpi
# web scraping from https://tradingeconomics.com/united-states/cpi-seasonally-adjusted
def cpi_forecast():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    result = requests.get('https://tradingeconomics.com/united-states/cpi-seasonally-adjusted', headers=headers)
    soup = BeautifulSoup(result.text, 'html.parser')

    css_cpi_forecast = 'td.datatable-item.datatable-item-positive'
    css_cpi_actual = css_cpi_forecast + '#actual'
    cpi_forcast = soup.select(css_cpi_forecast)[-1].text.split()
    cpi_actual = soup.select(css_cpi_actual)[-1].text.split()
    cpi_data = cpi_forcast if not cpi_actual else cpi_actual

    css_cpi_forecast_date = 'tr.an-estimate-row > td[style="text-align: left; padding-left: 10px; white-space: nowrap;"]'
    cpi_forecast_date = pd.to_datetime(soup.select(css_cpi_forecast_date)[1].text[:7] + '-01')

    css_cpi_publish_date = css_cpi_forecast_date
    cpi_publish_date = soup.select(css_cpi_forecast_date)[-1].text

    return cpi_forecast_date, float(cpi_data[0]), cpi_publish_date

In [9]:
# get forecast data for gdp
# web scarping from https://www.atlantafed.org/cqer/research/gdpnow/archives
def gdp_forecast(record=0):
    # record = 0 means the latest record
    # record = 1 means 1 earlier record than latest, 74 is the earliest record provided
    
    result = requests.get('https://www.atlantafed.org/cqer/research/gdpnow/archives')
    soup = BeautifulSoup(result.text, 'html.parser')
    
    dict_quarter_to_month = {'first': '01', 'second': '04', 'third': '07', 'fourth': '10'}
    css_gdp_forecast_archives = 'div[class="row GDPNowAllUpdates"] > p:nth-of-type(2)'
    gdp_forecast_archives = soup.select(css_gdp_forecast_archives)[record].text.split()
    
    quarter_index = gdp_forecast_archives.index('quarter')
    quarter_num = gdp_forecast_archives[quarter_index - 1]
    year = gdp_forecast_archives[quarter_index + 2]
    gdp_forecast_date = pd.to_datetime(year + '-' + dict_quarter_to_month[quarter_num] + '-01')

    percent_index = gdp_forecast_archives.index('percent')
    percent_num = gdp_forecast_archives[percent_index - 1]
    gdp_forecast_percent = float(percent_num)

    css_publish_date = 'div[class="row GDPNowAllUpdates"] > p:nth-of-type(1)'
    publish_date = soup.select(css_publish_date)[record].text
    
    return gdp_forecast_date, gdp_forecast_percent, publish_date

In [10]:
# functions to process quarterly data GDP into monthly data and fill na with forecasat percent
def check_na(series):
    # check number of na from last row to last row of valid value
    i = 1
    while pd.isna(series.iloc[-i]):  
        i += 1
    return i - 1
    
def fillna_with_percent_of_last_value(series, percent=0):
    # last row of na is filled by last row of valid value to a percent change
    series = series.copy()
    i = 1
    while pd.isna(series.iloc[-i]):  
        i += 1
    else:   
        if i != 1:
            series.iloc[-1] = round(series.iloc[-i] * (1 + percent / 100),3)
        return series

def fillna_with_two_adj_value(series):
    # calculate na from two adjacnet value by linear method, apply both from top and bottom of rows
    series = series.copy()
    for i in range(-1, 1):
        j = i
        while pd.isna(series.iloc[j]):  
            j = j + 1 - 2 * (i < 0)
        else:   
            if i != j:
                series.iloc[i] = series.iloc[j] * 2 - series.iloc[j + 1 - 2 * (i < 0)]
    return series

def process_gdp(series, percent = 0):
    # turn quarterly data to monthly data and fill na with forecast percent
    # call above 3 functions
    series = series.copy()
    na = check_na(series)
    shift = na - 3 - (na > 0)
    
    series_intrp_1 = series.interpolate(limit_direction='backward').shift(1)
    series_shift = series_intrp_1.shift(shift)
    series_percent = fillna_with_percent_of_last_value(series_shift, percent)
    series_intrp_2 = series_percent.interpolate(limit_direction='forward').shift(-shift)
    series_final = series_intrp_2.fillna(value=series_intrp_1)
    
    return fillna_with_two_adj_value(series_final)

In [11]:
# copy the dataframe
combine_cleanse = combine.copy()

# Get the forecast data for unrate, cpi and gpd
unrate_forecast_date, unrate_data, unrate_publish_date = unrate_forecast()
cpi_forecast_date, cpi_data, cpi_publish_date = cpi_forecast()
gdp_forecast_date, gdp_forecast_percent, gdp_publish_date = gdp_forecast()

# unrate and cpi forecast can be fillna directly
unrate_forecast_entry = pd.Series(unrate_data, index=[unrate_forecast_date])
cpi_forecast_entry = pd.Series(cpi_data, index=[cpi_forecast_date])
combine_cleanse['UNRATE'] = combine['UNRATE'].fillna(unrate_forecast_entry)
combine_cleanse['CPIAUCSL'] = combine['CPIAUCSL'].fillna(cpi_forecast_entry)

# gdp forecast percent have to go to several functions for calculation to fillna
combine_cleanse['GDP'] = process_gdp(combine['GDP'], gdp_forecast_percent)

In [12]:
# save cleaned data back to dictionary of dataframe
for df in dict_df:
    dict_df[df] = combine_cleanse[[df]].copy()

# 4. Save Data into Google BigQuery

In [13]:
# key is generated at Service Account and downloaded in json format
# default credentials is set under environment variable "GOOGLE_APPLICATION_CREDENTIALS"
# the value of "GOOGLE_APPLICATION_CREDENTIALS" is set to path of json e.g. '/Users/.../project_id.json'

# get project_id from json
credentials, project_id = google.auth.default()
database_id = 'data'

# save financial data into Google bigquery
for df_name in dict_df:
    table_id = df_name
    dict_df[df_name].to_gbq(f'{database_id}.{table_id}', project_id=project_id, if_exists='replace')

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4064.25it/s]


In [14]:
# Check if data can be accessed
client = bq.Client()
for df_name in dict_df:
    table_id = df_name
    sql = f'select * from {project_id}.{database_id}.{table_id}'
    # save result into dataframe
    display(client.query(sql).to_dataframe())

,date,FEDFUNDS
0,1961-08-01,2.00
1,2008-06-01,2.00
2,2008-08-01,2.00
3,1956-02-01,2.50
4,1956-03-01,2.50
...,...,...
832,1983-10-01,9.48
833,1980-05-01,10.98
834,2008-11-01,0.39
835,2016-07-01,0.39


,date,TB3MS
0,1956-03-01,2.25
1,1960-12-01,2.25
2,2018-10-01,2.25
3,2008-01-01,2.75
4,1993-01-01,3.00
...,...,...
832,1990-02-01,7.74
833,1990-05-01,7.74
834,1978-10-01,7.99
835,1979-03-01,9.48


,date,UNRATE
0,1968-04-01,3.5
1,1968-05-01,3.5
2,1968-08-01,3.5
3,1969-06-01,3.5
4,1969-07-01,3.5
...,...,...
832,2010-03-01,9.9
833,2010-04-01,9.9
834,1982-10-01,10.4
835,1983-01-01,10.4


,date,CPIAUCSL
0,1957-05-01,28.000
1,1959-02-01,29.000
2,1967-12-01,34.000
3,1971-03-01,40.000
4,1975-07-01,54.000
...,...,...
832,2023-09-01,307.288
833,2022-09-01,296.341
834,2014-11-01,236.983
835,2015-10-01,237.733


,date,GDP
0,1954-07-01,388.083333
1,1954-08-01,390.996000
2,1954-09-01,393.908667
3,1954-10-01,396.821333
4,1954-11-01,399.734000
...,...,...
832,2023-11-01,27956.998000
833,2023-12-01,28189.973000
834,2024-01-01,28422.948000
835,2024-02-01,28655.923000
